In [1]:
#import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
from citipy import citipy
#import api key from file
from apikey import googlekey

# Configure gmaps
gmaps.configure(api_key=googlekey)

In [6]:
csvpath = os.path.join("..","Data","forecast_data.csv")

# read in WeatherPy forecast data CSV file
citydata = pd.read_csv(csvpath)
display(citydata)

,Unnamed: 0,City Name,Latitude,Longitude,Country,UTC +/- (hours),Max Temperature,Humidity,Cloudiness,Wind Speed
0,0,Codrington,-38.2667,141.9667,AU,10.0,69.89,53,81,13.67
1,1,Mataura,-46.1927,168.8643,NZ,12.0,59.16,66,46,5.03
2,2,Rikitea,-23.1203,-134.9692,PF,-9.0,78.93,84,95,24.34
3,3,Saint Paul Harbor,57.7900,-152.4072,US,-8.0,44.60,68,100,14.03
4,4,Ushuaia,-54.8000,-68.3000,AR,-3.0,42.06,84,92,3.38
...,...,...,...,...,...,...,...,...,...,...
549,549,Vilyuysk,63.7553,121.6247,RU,9.0,39.25,90,100,7.74
550,550,Puerto del Rosario,28.5004,-13.8627,ES,1.0,68.58,54,31,18.72
551,551,La Providencia,28.7167,-111.5833,MX,-7.0,89.91,23,4,15.37
552,552,Yelizovo,53.1873,158.3805,RU,12.0,35.92,94,94,3.83


In [8]:
# plot heatmap of city humidity on google maps
citylocs = citydata[["Latitude","Longitude"]]
humidity = citydata["Humidity"]

fig = gmaps.figure()

humidity_layer = gmaps.heatmap_layer(citylocs,weights=humidity,
                                     dissipating=False,max_intensity =100,
                                     point_radius=1)
fig.add_layer(humidity_layer)
display(fig)

Figure(layout=FigureLayout(height='420px'))

In [36]:
# filter city data to find cities with ideal weather conditions
idealvacation = citydata.copy()[(citydata["Humidity"] < 40)
                         & (citydata["Max Temperature"]< 75)
                         & (citydata["Max Temperature"]> 60)
                         & (citydata["Cloudiness"] < 50)
                         & (citydata["Wind Speed"]>10)
                         & (citydata["Wind Speed"]<20)
                         & (citydata["Country"] == "US") ]

display(idealvacation)

,Unnamed: 0,City Name,Latitude,Longitude,Country,UTC +/- (hours),Max Temperature,Humidity,Cloudiness,Wind Speed
97,97,Wilmington,34.2257,-77.9447,US,-4.0,73.54,17,2,13.65
293,293,Hamilton,39.1834,-84.5333,US,-4.0,67.39,34,1,11.61
450,450,Havelock,34.8791,-76.9013,US,-4.0,71.28,25,42,15.64
508,508,Philadelphia,39.9523,-75.1638,US,-4.0,64.45,37,6,17.31


In [72]:
idealvacation["Hotel Name"] = ""
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 

for index,row in idealvacation.iterrows():
    target_coordinates = f"{row['Latitude']}, {row['Longitude']}"
    target_search = "Hotel"
    target_radius = 5000 # Note: this is in meters
    target_type = "lodging"
    
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": googlekey
    }

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    hotel = response["results"][0]["name"]
    idealvacation.loc[index,"Hotel Name"] = hotel
    print(f"One hotel in {row['City Name']} is {hotel}")


One hotel in Wilmington is Hotel Ballast Wilmington, Tapestry Collection by Hilton
One hotel in Hamilton is Graduate Cincinnati
One hotel in Havelock is Hampton Inn Havelock
One hotel in Philadelphia is Cambria Hotel Downtown-Center City


In [38]:
# Verify data aquisition
display(idealvacation)

,Unnamed: 0,City Name,Latitude,Longitude,Country,UTC +/- (hours),Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
97,97,Wilmington,34.2257,-77.9447,US,-4.0,73.54,17,2,13.65,"Hotel Ballast Wilmington, Tapestry Collection ..."
293,293,Hamilton,39.1834,-84.5333,US,-4.0,67.39,34,1,11.61,Graduate Cincinnati
450,450,Havelock,34.8791,-76.9013,US,-4.0,71.28,25,42,15.64,Hampton Inn Havelock
508,508,Philadelphia,39.9523,-75.1638,US,-4.0,64.45,37,6,17.31,Cambria Hotel Downtown-Center City


In [70]:
# Create hotel pin layer
hotelpins = gmaps.symbol_layer(
    idealvacation[["Latitude","Longitude"]].astype(float), fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=20,
    info_box_content=[f"Hotel Name: {hotel}" for hotel in idealvacation["Hotel Name"] + "<br /> "
                     +"City: " + idealvacation["City Name"] + "<br />"
                     + "Country: " + idealvacation["Country"]]
)

In [71]:
# plot final map
fig = gmaps.figure()
fig.add_layer(humidity_layer)
fig.add_layer(hotelpins)

fig

Figure(layout=FigureLayout(height='420px'))